This notebook was used to get reasonable initial values for EPR spin-Hamiltonian parameters prior to EasySpin simulations.

Written by Felippe M. Colombari - LNBR/CNPEM (felippe.colombari@lnbr.cnpem.br)

May, 2024

In [ ]:
#%pip install matplotlib pandas numpy scipy
import pandas as pd
import numpy as np 
import scipy as sp 
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

In [ ]:
colnames=["freq", "signal"]

In [ ]:
spectrum = pd.read_csv("../demo/EPR_WT_avicel.txt", sep=",\t", names=colnames, header=0, index_col=False)

In [ ]:
x_data = spectrum['freq']
y_data = spectrum['signal']

In [ ]:
peaks_pos, properties_pos = find_peaks(  y_data, height=862, width=5, prominence=60, rel_height=0.05 )
peaks_neg, properties_neg = find_peaks( -y_data, height=100, width=5, prominence=200, rel_height=0.05 )

In [ ]:
all_peaks = np.concatenate((peaks_pos, peaks_neg))

In [ ]:
all_properties = np.stack((properties_pos, properties_neg), casting='same_kind')

In [ ]:
h = 6.626E-34         # J.s
f = 9.14E9            # Hz ou 1/s
mB = 9.2740154E-24    # J/T
gauss2mhz = 1/0.35683 # conversion factor

In [ ]:
x_data_scale = h * f / (mB * x_data/1000)

Get initial value for A|| by estimating the average distance between peaks at this region:

In [ ]:
A_par = [0] * 3
A_par[0] = x_data[all_peaks[1]] - x_data[all_peaks[0]]
A_par[1] = x_data[all_peaks[2]] - x_data[all_peaks[1]]
A_par[2] = x_data[all_peaks[3]] - x_data[all_peaks[2]]

A_par_center = (x_data[all_peaks[3]] + x_data[all_peaks[0]])/2

A_parallel_mT = (x_data[all_peaks[3]] - x_data[all_peaks[0]])/3
A_parallel_G = A_parallel_mT * 10
A_parallel_MHz = A_parallel_G * gauss2mhz

print("A||  : %6.2f mT, or %6.2f G, or %6.3f MHz" % (A_parallel_mT, A_parallel_G, A_parallel_MHz))

Use derivatives to estimate the number of shoulders at the perpendicular region:

In [ ]:
der2 = savgol_filter(y_data, window_length=10, polyorder=4, deriv=2)
max_der2 = np.max(np.abs(der2))
large = np.where(np.abs(der2) > max_der2/2)[0]
gaps = np.diff(large) > 10
begins = np.insert(large[1:][gaps], 0, large[0])
ends = np.append(large[:-1][gaps], large[-1])
changes = ((begins+ends)/2).astype(int)

new_changes = [ x for x in changes if x > peaks_pos[-1] and x < peaks_neg[-1] ]

n_A_per = len(new_changes) + 2 # add two points corresponding to first (positive) and last (negative) peaks

print("Number of hyperfine at perpendicular region is %i " % n_A_per)

And get initial value for A_|_ by estimating the average distance between shoulders:

In [ ]:
A_per_mT = (x_data[all_peaks[5]] - x_data[all_peaks[4]])/ n_A_per
A_per_G = A_per_mT * 10
A_per_MHz = A_per_G * gauss2mhz

A_per_center = (x_data[all_peaks[5]] + x_data[all_peaks[4]])/2

print("A_|_ : %6.2f mT, or %6.2f G, or %6.2f MHz" % (A_per_mT, A_per_G, A_per_MHz))

In [ ]:
import matplotlib.ticker as ticker

fig, ax1 = plt.subplots()
fig.set_size_inches(10,5)

plt.axis([220, 370, -9900, 9500])

ax1.set_title('dataset #6 - WT + avicel')

ax1.plot(x_data, y_data, label="Raw data", lw=1.7)
ax1.plot(x_data[peaks_pos], y_data[peaks_pos], 'r+', ms=10, label="Detected peaks")
ax1.plot(x_data[peaks_neg], y_data[peaks_neg], 'r+', ms=10, label="Detected peaks")

ax1.grid(which='major', color='black', linewidth=0.3)
ax1.grid(which='minor', color='black', linewidth=0.1)
ax1.minorticks_on()
ax1.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax1.xaxis.set_minor_locator(ticker.MultipleLocator(2))

ax1.set_xlabel('freq (mT)')
ax1.set_ylabel('signal')

plt.axhline(y=0., color='black', linestyle='-.', lw=0.6)

idx = np.argwhere(np.diff(np.sign(y_data - 0))).flatten()
ax1.plot(x_data[idx], y_data[idx], 'r+', ms=10)

#A parallel stuff
px0 = [x_data[all_peaks[0]], x_data[all_peaks[0]]]
px1 = [x_data[all_peaks[1]], x_data[all_peaks[1]]]
px2 = [x_data[all_peaks[2]], x_data[all_peaks[2]]]
px3 = [x_data[all_peaks[3]], x_data[all_peaks[3]]]
px4 = [x_data[all_peaks[0]], x_data[all_peaks[3]]]
py1 = [-1200, -800]
py2 = [-1200, -1200]

ax1.plot(px0, py1, color='black', linestyle='-')
ax1.plot(px1, py1, color='black', linestyle='-')
ax1.plot(px2, py1, color='black', linestyle='-')
ax1.plot(px3, py1, color='black', linestyle='-')
ax1.plot(px4, py2, color='black', linestyle='-')

ax1.text(A_par_center, -2200, 
         "A$_{\parallel}$ = " + str(f'%5.1f' % A_parallel_mT) + " mT " + str(f'(%5.1f' % A_parallel_MHz) + " MHz)", 
         fontsize = 12, va='center', ha='center', fontfamily='sans-serif',
         bbox = dict(facecolor = 'white', alpha = 0.9))

#A perpendicular stuff
px0 = [x_data[all_peaks[4]], x_data[all_peaks[4]]]
px1 = [x_data[new_changes[1]], x_data[new_changes[1]]]
px2 = [x_data[new_changes[2]], x_data[new_changes[2]]]
px3 = [x_data[new_changes[3]], x_data[new_changes[3]]]
px4 = [x_data[new_changes[0]], x_data[new_changes[0]]]
pxh = [x_data[all_peaks[5]], x_data[all_peaks[4]]]
pxf = [x_data[all_peaks[5]], x_data[all_peaks[5]]]

py1 = [4200, 3800]
py2 = [4200, 4200]

ax1.plot(px0, py1, color='black', linestyle='-')
ax1.plot(px1, py1, color='black', linestyle='-')
ax1.plot(px2, py1, color='black', linestyle='-')
ax1.plot(px3, py1, color='black', linestyle='-')
ax1.plot(px4, py1, color='black', linestyle='-')

ax1.plot(pxf, py1, color='black', linestyle='-')
ax1.plot(pxh, py2, color='black', linestyle='-')

ax1.text(A_per_center, 5200, 
         "A$_{\perp}$ = " + str(f'%5.1f' % A_per_mT) + " mT " + str(f'(%5.1f' % A_per_MHz) + " MHz)",
         fontsize = 12, va='center', ha='center', fontfamily='sans-serif',
         bbox = dict(facecolor = 'white', alpha = 0.9))


ax1.plot(x_data[new_changes], y_data[new_changes], 'rx', ms=5)

plt.savefig('fig_dataset03_A.png', dpi=300, pad_inches=0.1, bbox_inches='tight')
plt.show()

Get g|| by estimating the center of the parallel region:

In [ ]:
g_parallel = (x_data_scale[all_peaks[0]] + x_data_scale[all_peaks[3]]) / 2
print("g_parallel: %s" % (g_parallel))

Get g_|_ by estimating the center of the perpendicular region:

In [ ]:
g_per = (x_data_scale[all_peaks[5]] + x_data_scale[all_peaks[4]]) / 2
print("g_perpendicular: %s" % (g_per))

In [ ]:
fig, ax1 = plt.subplots()
fig.set_size_inches(10,5)

plt.axis([2.75, 1.80, -9500, 9500])

ax1.set_title('dataset #6 - WT + avicel')

ax1.plot(x_data_scale, y_data, label="Raw data", lw=1.5)
ax1.plot(x_data_scale[peaks_pos], y_data[peaks_pos], 'r+', ms=10, label="Detected peaks")
ax1.plot(x_data_scale[peaks_neg], y_data[peaks_neg], 'r+', ms=10, label="Detected peaks")

ax1.grid(which='major', color='black', linewidth=0.3)
ax1.grid(which='minor', color='black', linewidth=0.1)
ax1.minorticks_on()
ax1.xaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax1.xaxis.set_minor_locator(ticker.MultipleLocator(0.01))

ax1.set_xlabel('g-factor')
ax1.set_ylabel('intensity')

plt.axhline(y=0., color='black', linestyle='-.', lw=0.6)

ax1.plot(x_data_scale[idx], y_data[idx], 'r+')

#g parallel stuff
px1 = [g_parallel, g_parallel]
py1 = [-1200, -800]

ax1.plot(px1, py1, color='black', linestyle='-')

ax1.text(g_parallel, -2100, 
         "g$_{\parallel}$ = " + str(f'%7.5f' % g_parallel), 
         fontsize = 12, va='center', ha='center', fontfamily='sans-serif',
         bbox = dict(facecolor = 'white', alpha = 0.9))

#g perpendicular stuff
px1 = [g_per, g_per]
py1 = [4600, 4200]

ax1.plot(px1, py1, color='black', linestyle='-')

ax1.text(g_per, 5500, 
         "g$_{\perp}$ = " + str(f'%7.5f' % g_per), 
         fontsize = 12, va='center', ha='center', fontfamily='sans-serif',
         bbox = dict(facecolor = 'white', alpha = 0.9))

plt.savefig('fig_dataset03_g.png', dpi=300, pad_inches=0.1, bbox_inches='tight')
plt.show()

In [ ]:
print("These are the position of all peaks: %s mT" % (np.array(x_data[all_peaks])))

In [ ]:
print("These are the position of all peaks: %s (g_factor)" % (np.array(x_data_scale[all_peaks])))